In [ ]:
from farm.infer import Inferencer
import glob
import pandas as pd
import tqdm

In [ ]:
questions_input = [
    "When did the operator propose to speak again to customer?",
    "When did the operator propose to call back again?",
    "Did the customer agree to speak again?",
    "What date was set for another meeting",
    "What date is set for the next meeting?",
    "What date is set for another meeting?",
    "What is the date of the next meeting?",
    "Is a follow up call proposed in the future?",
    "Is a meeting proposed in the future?",
    "Did the operator propose to keep in touch?",
    "Did the customer provide an email address?",
]

model_input = []
call_ids = []
files = glob.glob("data/text/*.txt")
for file in files:
    call_id = file.split("/")[-1].split(".")[0]
    with open(file, "r") as f:
        text = f.readlines()
    joined_text = ''.join(text)
    model_input.append({
        "questions": questions_input,
        "text": joined_text,
    })
    call_ids.append(call_id)

In [ ]:
model_name = "deepset/xlm-roberta-large-squad2"
model_max_seq_len = 384
model_doc_stride = 128
nlp = Inferencer.load(model_name, task_type="question_answering", gpu=True, max_seq_len=model_max_seq_len, doc_stride=model_doc_stride)

02/22/2023 21:05:28 - INFO - farm.utils -   Using device: CUDA 
02/22/2023 21:05:28 - INFO - farm.utils -   Number of GPUs: 1
02/22/2023 21:05:28 - INFO - farm.utils -   Distributed Training: False
02/22/2023 21:05:28 - INFO - farm.utils -   Automatic Mixed Precision: None
02/22/2023 21:05:28 - INFO - farm.modeling.language_model -   
02/22/2023 21:05:28 - INFO - farm.modeling.language_model -   LOADING MODEL
02/22/2023 21:05:28 - INFO - farm.modeling.language_model -   =============
02/22/2023 21:05:28 - INFO - farm.modeling.language_model -   Could not find deepset/xlm-roberta-large-squad2 locally.
02/22/2023 21:05:28 - INFO - farm.modeling.language_model -   Looking on Transformers Model Hub (in local cache and online)...
Some weights of the model checkpoint at deepset/xlm-roberta-large-squad2 were not used when initializing XLMRobertaModel: ['qa_outputs.bias', 'qa_outputs.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on a

In [17]:
model_output = nlp.inference_from_dicts(dicts=model_input)

Inferencing Samples: 100%|██████████| 105/105 [00:31<00:00,  3.35 Batches/s]


In [35]:
dataframe = pd.DataFrame(columns=["model", "conversation_id", "question_ref", "answer_ref", "question", "answer", "score", "offset_answer_start", "offset_answer_end", "context", "offset_context_start", "offset_context_end"])

for task in (model_output):
    predictions = task["predictions"][0]
    
    conversation_ref = int(predictions["id"].split("-")[0])
    conversation_id = call_ids[conversation_ref]
    
    question_ref = int(predictions["id"].split("-")[-1])
    question = predictions["question"]
    
    answer_ref = 0
    
    for answer_data in predictions["answers"]:
        answer = answer_data["answer"]
        score = answer_data["score"]
        offset_answer_start = answer_data["offset_answer_start"]
        offset_answer_end = answer_data["offset_answer_end"]
        context = answer_data["context"]
        offset_context_start = answer_data["offset_context_start"]
        offset_context_end = answer_data["offset_context_end"]
        
        dataframe = pd.concat([dataframe, pd.Series([model_name, conversation_id, question_ref, answer_ref, question, answer, score, offset_answer_start, offset_answer_end, context, offset_context_start, offset_context_end])])

KeyboardInterrupt: 

In [ ]:
dataframe.head(20)